In [ ]:
import os
import ansys.speos.script as script
from ansys.speos.core import Speos

tests_data_path = os.path.join(os.path.abspath(''), os.path.pardir, "assets")

Create connection with speos rpc server

In [ ]:
speos = Speos(host="localhost", port=50051)

Load a speos file into a project feature.

In [ ]:
p = script.Project(speos=speos, path=os.path.join(tests_data_path, "Prism.speos", "Prism.speos"))
print(p)

In [ ]:
import pyvista as pv
import numpy as np
import ansys.speos.core as core
from google.protobuf.internal.containers import RepeatedScalarFieldContainer

def __extract_part_mesh_info(part_data: core.Part, part_coordinate_info: RepeatedScalarFieldContainer = None) -> pv.PolyData:
        """
        extract mesh data info from a part.

        Parameters
        ----------
        part_data: ansys.api.speos.part.v1.part_pb2
            Part from scene.
        part_coordinate_info: RepeatedScalarFieldContainer
            message contains part coordinate info: origin, x_vector, y_vector, z_vector
        Returns
        -------
        pv.PolyData
            mesh data extracted.

        """

        def local2absolute(local_vertice: np.ndarray) -> np.ndarray:
            """
            convert local coordinate to global coordinate.

            Parameters
            ----------
            local_vertice: np.ndarray
                numpy array includes x, y, z info.

            Returns
            -------
            np.ndarray
             numpy array includes x, y, z info

            """
            global_origin = np.array(part_coordinate.origin)
            global_x = np.array(part_coordinate.x_vect) * local_vertice[0]
            global_y = np.array(part_coordinate.y_vect) * local_vertice[1]
            global_z = np.array(part_coordinate.z_vect) * local_vertice[2]
            return global_origin + global_x + global_y + global_z

        part_coordinate = core.AxisSystem()
        part_coordinate.origin = [0.0, 0.0, 0.0]
        part_coordinate.x_vect = [1.0, 0.0, 0.0]
        part_coordinate.y_vect = [0.0, 1.0, 0.0]
        part_coordinate.z_vect = [0.0, 0.0, 1.0]
        if part_coordinate_info is not None:
            part_coordinate.origin = part_coordinate_info[:3]
            part_coordinate.x_vect = part_coordinate_info[3:6]
            part_coordinate.y_vect = part_coordinate_info[6:9]
            part_coordinate.z_vect = part_coordinate_info[9:]
        part_mesh_info = None
        for body_idx, body_guid in enumerate(part_data.body_guids):
            body_item_data = speos.client.get_item(body_guid).get()
            for face_idx, face_guid in enumerate(body_item_data.face_guids):
                face_item_data = speos.client.get_item(face_guid).get()
                vertices = np.array(face_item_data.vertices)
                facets = np.array(face_item_data.facets)
                vertices = vertices.reshape(-1, 3)
                vertices = np.array([local2absolute(vertice) for vertice in vertices])
                facets = facets.reshape(-1, 3)
                temp = np.full(facets.shape[0], 3)
                temp = np.vstack(temp)
                facets = np.hstack((temp, facets))
                face_mesh_data = pv.PolyData(vertices, facets)
                if part_mesh_info is None:
                    part_mesh_info = face_mesh_data
                else:
                    part_mesh_info = part_mesh_info.append_polydata(face_mesh_data)
        return part_mesh_info

def preview(project: script.Project) -> None:
    """preview cad bodies inside the scene."""

    p = pv.Plotter()

    _preview_mesh = None
    
    root_part_data = speos.client.get_item(project.scene_link.get().part_guid).get()
    if len(root_part_data.parts) != 0:
        for part_idx, part_item in enumerate(root_part_data.parts):
            part_item_data = speos.client.get_item(part_item.part_guid).get()
            if _preview_mesh is None:
                _preview_mesh = __extract_part_mesh_info(part_item_data, part_item.axis_system)
            else:
                _preview_mesh = _preview_mesh.append_polydata(
                    __extract_part_mesh_info(part_item_data, part_item.axis_system)
                )
    else:
        _preview_mesh = __extract_part_mesh_info(root_part_data)

    p.add_mesh(_preview_mesh, show_edges=True)
    p.show()

In [ ]:
preview(project=p)

Compute

In [ ]:
sim = p.find(name="Prism", feature_type=script.Simulation)
sim.compute_CPU()

Watch results

In [ ]:
from ansys.speos.workflow.open_result import open_result_image

open_result_image(simulation_feature=sim, result_name="Prism.Irradiance.1.xmp")

Modify irradiance sensor - compute - watch result

Here the irradiance sensor is modified regarding its wavelengths range : from 500 to 600 nm.  
Then the simulation is computed again, and we can see the difference in the output xmp.

In [ ]:
irr = p.find(name="Irradiance.1:564")
irr.set_irradiance().set_type_spectral().set_wavelengths_range().set_start(500).set_end(600).set_sampling(11)
irr.commit()

sim.compute_CPU()

open_result_image(simulation_feature=sim, result_name="Prism.Irradiance.1.xmp")